# 패턴인식 (21_2)
## 휴먼지능정보공학과 201810793 이연우
### [과제#2] 히스토그램 역투영
첨부한 베이스 코드를 기반으로 히스토그램 역투영 코드를 완성하세요.

추가로 모폴로지 연산을 적용하여 히스토그램 역투영 결과를 개선시켜보세요. 

※ 참고로 양자화 관련 코드는 구현하지 않아도 되며, 코드를 완성한 후 input 영상에 새로운 손 이미지를 대입하여 히스토그램 역투영 알고리즘이 피부색을 잘 찾아내는지 테스트해보세요

In [1]:
import cv2
import numpy as np

- 모델 영상의 2차원 H-S 히스토그램 계산

In [2]:
img_m = cv2.imread('model.jpg')
hsv_m = cv2.cvtColor(img_m, cv2.COLOR_BGR2HSV)
hist_m = cv2.calcHist([hsv_m], [0, 1], None, [180, 256], [0, 180, 0, 256])

- 입력 영상의 2차원 H-S 히스토그램 계산

In [3]:
img_i = cv2.imread('hand.jpg')
hsv_i = cv2.cvtColor(img_i, cv2.COLOR_BGR2HSV)
hist_i = cv2.calcHist([hsv_i], [0, 1], None, [180, 256], [0, 180, 0, 256])

- 히스토그램 정규화
    
    각 픽셀 개수 / 전체 픽셀 값

In [4]:
# HW ------
hist_m_nor = hist_m / (img_m.shape[0] * img_m.shape[1])         
print("maxinum of hist_m_nor : % f" % hist_m_nor.max()) # 값 범위 체크 : 1.0 이하

hist_i_nor = hist_i / (img_i.shape[0] * img_i.shape[1])
#cv2.normalize(hist_i, hist_i, 0, 1, cv2.NORM_MINMAX)
print("maxinum of hist_i : % f" % hist_i_nor.max()) # 값 범위 체크 : 1.0 이하
#-----------

maxinum of hist_m_nor :  0.021333
maxinum of hist_i :  0.685269


- 비율 히스토그램 계산

In [5]:
# HW ------
hist_r = np.minimum(hist_m / (hist_i + 1e-7), 1.0)
#-----------
print("range of hist_r : [%.lf, %.lf]" % (hist_r.min(), hist_r.max()))
# 비율 값 범위 체크 : [0.0, 1.0]

range of hist_r : [0, 1]


- 히스토그램 역투영 수행

In [6]:
# HW ------
height, width = img_i.shape[0], img_i.shape[1]
result = np.zeros((height, width))
h, s, v = cv2.split(hsv_i)

for w in range(width):
    for he in range(height):
        result[he, w] = hist_r[h[he,w], s[he,w]]
#-----------

- 이진화 수행 (화소값이 임계값 0.02보다 크면 255, 그렇지 않으면 0)

In [7]:
ret, thresholded = cv2.threshold(result, 0.02, 255, cv2.THRESH_BINARY)
cv2.imwrite('result.jpg', thresholded)

True

- 모폴로지 연산 적용

In [11]:
# HW ------
kernel = np.ones((13, 13), np.uint8)
improved = cv2.morphologyEx(thresholded, cv2.MORPH_CLOSE, kernel)
#-----------
cv2.imwrite('morphology.jpg', improved)

True